# Data Aggregation and Group Operations

In [2]:
import numpy as np 
import pandas as pd

## How to think about Group operations

In [3]:
frame = pd.DataFrame({
    "key": ['a','a', None, 'b','b','a', None], 
    "key2": pd.Series([1,2,1,2,1,None,1], dtype="Int64"),
    "data1": np.random.standard_normal(7),
    "data2": np.random.standard_normal(7),
})

frame

,key,key2,data1,data2
0,a,1,0.315934,0.249057
1,a,2,0.481413,-2.092307
2,None,1,-0.023367,-0.400921
3,b,2,0.318452,-0.169628
4,b,1,-1.393540,0.137534
5,a,<NA>,1.937521,1.767177
6,None,1,0.202862,-0.665536


In [5]:
grouped = frame['data1'].groupby(frame['key'])
grouped.mean()

key
a    0.911622
b   -0.537544
Name: data1, dtype: float64

In [8]:
means = frame['data1'].groupby([frame['key'], frame['key2']]).mean()
means

key  key2
a    1       0.315934
     2       0.481413
b    1      -1.393540
     2       0.318452
Name: data1, dtype: float64

In [9]:
means.unstack()

key2,1,2
key,,
a,0.315934,0.481413
b,-1.393540,0.318452


In [11]:
states = np.array(['oh', 'ca', 'ca', 'oh', 'oh', 'ca', 'oh'])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

# Same as concatenating two new columns and then grouping by their names
frame['data1'].groupby([states, years]).mean()

ca  2005    1.209467
    2006   -0.023367
oh  2005    0.317193
    2006   -0.595339
Name: data1, dtype: float64

In [16]:
frame.groupby('key').mean(),frame.groupby('key2').mean(),frame.groupby(['key', 'key2']).mean()

C:\Users\William Costa\AppData\Local\Temp\ipykernel_11092\2336427859.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  frame.groupby('key').mean(),frame.groupby('key2').mean(),frame.groupby(['key', 'key2']).mean()


(     key2     data1     data2
 key                          
 a     1.5  0.911622 -0.025358
 b     1.5 -0.537544 -0.016047,
          data1     data2
 key2                    
 1    -0.224528 -0.169967
 2     0.399933 -1.130967,
              data1     data2
 key key2                    
 a   1     0.315934  0.249057
     2     0.481413 -2.092307
 b   1    -1.393540  0.137534
     2     0.318452 -0.169628)

In [18]:
frame.groupby(['key', 'key2']).size(), frame.groupby(['key', 'key2'],dropna=False).size()

(key  key2
 a    1       1
      2       1
 b    1       1
      2       1
 dtype: int64,
 key  key2
 a    1       1
      2       1
      <NA>    1
 b    1       1
      2       1
 NaN  1       2
 dtype: int64)

In [21]:
frame.groupby('key').count()

,key2,data1,data2
key,,,
a,2,3,3
b,2,2,2


## Iterating over Groups

In [22]:
for name, group in frame.groupby('key'):
    print(name)
    print(group)

a
  key  key2     data1     data2
0   a     1  0.315934  0.249057
1   a     2  0.481413 -2.092307
5   a  <NA>  1.937521  1.767177
b
  key  key2     data1     data2
3   b     2  0.318452 -0.169628
4   b     1 -1.393540  0.137534


In [23]:
for (key1, key2), group in frame.groupby(['key', 'key2']):
    print((key1,key2))
    print(group)

('a', 1)
  key  key2     data1     data2
0   a     1  0.315934  0.249057
('a', 2)
  key  key2     data1     data2
1   a     2  0.481413 -2.092307
('b', 1)
  key  key2    data1     data2
4   b     1 -1.39354  0.137534
('b', 2)
  key  key2     data1     data2
3   b     2  0.318452 -0.169628


In [24]:
pieces = {name: group for name, group in frame.groupby('key')}
pieces

{'a':   key  key2     data1     data2
 0   a     1  0.315934  0.249057
 1   a     2  0.481413 -2.092307
 5   a  <NA>  1.937521  1.767177,
 'b':   key  key2     data1     data2
 3   b     2  0.318452 -0.169628
 4   b     1 -1.393540  0.137534}

In [25]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

a
0    0.315934
1    0.481413
5    1.937521
Name: data1, dtype: float64
b
3    0.318452
4   -1.393540
Name: data1, dtype: float64


## Grouping with Dictionaries and Series

In [26]:
people = pd.DataFrame(np.random.standard_normal((5,5)), columns=list('abcde'), index=['Joe', 'Steve', 'Wanda', 'Jill', 'Trey'])

people.iloc[2:3, [1, 2]] = np.nan

people

,a,b,c,d,e
Joe,1.690448,-1.690977,-1.839546,-0.993657,0.350033
Steve,-0.187892,-0.936909,-0.438740,0.623314,0.942163
Wanda,0.542597,NaN,NaN,0.341314,-0.623878
Jill,0.935886,-1.086220,-0.696052,-0.662298,1.229252
Trey,0.422720,-0.267345,-1.132960,0.766927,-2.554107


In [27]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}
by_column = people.groupby(mapping, axis='columns')

by_column.sum()

,blue,red
Joe,-2.833203,0.349504
Steve,0.184574,-0.182638
Wanda,0.341314,-0.081281
Jill,-1.358351,1.078918
Trey,-0.366032,-2.398732


In [28]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [29]:
people.groupby(map_series, axis='columns').count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


## Grouping with Functions

In [30]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.690448,-1.690977,-1.839546,-0.993657,0.350033
4,1.358606,-1.353565,-1.829012,0.104629,-1.324855
5,0.354704,-0.936909,-0.438740,0.964628,0.318285


In [38]:
key_list = ['one', 'two', 'three', 'four', 'five']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one    1.690448 -1.690977 -1.839546 -0.993657  0.350033
4 five   0.422720 -0.267345 -1.132960  0.766927 -2.554107
  four   0.935886 -1.086220 -0.696052 -0.662298  1.229252
5 three  0.542597       NaN       NaN  0.341314 -0.623878
  two   -0.187892 -0.936909 -0.438740  0.623314  0.942163

## Grouping by Index Level

In [41]:
columns = pd.MultiIndex.from_arrays([
    ['us','us','us','jp','jp'],
    [1,3,5,1,3]],
    names=['city', 'tenor']
)
hierarchical_frame = pd.DataFrame(np.random.standard_normal((4,5)), columns = columns)
hierarchical_frame

city         us                            jp          
tenor         1         3         5         1         3
0     -0.352796  0.759863  0.627868 -0.677090 -0.379752
1      0.021500  0.746004  0.710238  0.597283 -1.726483
2      2.028633 -0.231347  0.192871  0.520016 -1.028707
3     -0.340201 -0.741973 -0.377101  1.171272 -1.021960

In [42]:
hierarchical_frame.groupby(level='city', axis='columns').count()

city,jp,us
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

In [44]:
grouped = frame.groupby('key')
grouped['data1'].min(),grouped['data1'].nsmallest(2)

(key
 a    0.315934
 b   -1.393540
 Name: data1, dtype: float64,
 key   
 a    0    0.315934
      1    0.481413
 b    4   -1.393540
      3    0.318452
 Name: data1, dtype: float64)

In [45]:
aggregation_function = lambda array: array.max() - array.min()

grouped.agg(aggregation_function)

,key2,data1,data2
key,,,
a,1,1.621587,3.859485
b,1,1.711992,0.307162


In [46]:
grouped.describe()

key2                                           data1            ...  \
    count mean       std  min   25%  50%   75%  max count      mean  ...   
key                                                                  ...   
a     2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.911622  ...   
b     2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.537544  ...   

                        data2                                          \
          75%       max count      mean       std       min       25%   
key                                                                     
a    1.209467  1.937521   3.0 -0.025358  1.944321 -2.092307 -0.921625   
b   -0.109546  0.318452   2.0 -0.016047  0.217196 -0.169628 -0.092837   

                                   
          50%       75%       max  
key                                
a    0.249057  1.008117  1.767177  
b   -0.016047  0.060744  0.137534  

[2 rows x 24 columns]

## Column-Wise and Multiple Function Application

In [47]:
location = r"C:\Users\William Costa\Documents\repositories\data_science_basic_knowledge\resources\tips.csv"
tips = pd.read_csv(location)

tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [48]:
tips['tips_pct'] = tips['tip'] / tips['total_bill']

tips.head()

,total_bill,tip,smoker,day,time,size,tips_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [50]:
grouped = tips.groupby(['day', 'smoker'])

grouped_pct = grouped['tips_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tips_pct, dtype: float64

In [51]:
grouped_pct.agg(['mean', 'std', aggregation_function])

mean       std  <lambda_0>
day  smoker                                
Fri  No      0.151650  0.028123    0.067349
     Yes     0.174783  0.051293    0.159925
Sat  No      0.158048  0.039767    0.235193
     Yes     0.147906  0.061375    0.290095
Sun  No      0.160113  0.042347    0.193226
     Yes     0.187250  0.154134    0.644685
Thur No      0.160298  0.038774    0.193350
     Yes     0.163863  0.039389    0.151240

In [52]:
grouped_pct.agg(['mean', 'std', ("max-min", aggregation_function)])

mean       std   max-min
day  smoker                              
Fri  No      0.151650  0.028123  0.067349
     Yes     0.174783  0.051293  0.159925
Sat  No      0.158048  0.039767  0.235193
     Yes     0.147906  0.061375  0.290095
Sun  No      0.160113  0.042347  0.193226
     Yes     0.187250  0.154134  0.644685
Thur No      0.160298  0.038774  0.193350
     Yes     0.163863  0.039389  0.151240

In [54]:
functions = ['count', 'mean', 'max']
result = grouped[['tips_pct', 'total_bill']].agg(functions)
result

tips_pct                     total_bill                  
               count      mean       max      count       mean    max
day  smoker                                                          
Fri  No            4  0.151650  0.187735          4  18.420000  22.75
     Yes          15  0.174783  0.263480         15  16.813333  40.17
Sat  No           45  0.158048  0.291990         45  19.661778  48.33
     Yes          42  0.147906  0.325733         42  21.276667  50.81
Sun  No           57  0.160113  0.252672         57  20.506667  48.17
     Yes          19  0.187250  0.710345         19  24.120000  45.35
Thur No           45  0.160298  0.266312         45  17.113111  41.19
     Yes          17  0.163863  0.241255         17  19.190588  43.11

In [55]:
functions = [('Average', 'mean'), ('Variance', np.var)]
result = grouped[['tips_pct', 'total_bill']].agg(functions)
result

tips_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [58]:
grouped.agg({'tip': np.max, 'size': 'sum'})

(               tip  size
 day  smoker             
 Fri  No       3.50     9
      Yes      4.73    31
 Sat  No       9.00   115
      Yes     10.00   104
 Sun  No       6.00   167
      Yes      6.50    49
 Thur No       6.70   112
      Yes      5.00    40,
      total_bill   tip smoker   day    time  size  tips_pct
 0         16.99  1.01     No   Sun  Dinner     2  0.059447
 1         10.34  1.66     No   Sun  Dinner     3  0.160542
 2         21.01  3.50     No   Sun  Dinner     3  0.166587
 3         23.68  3.31     No   Sun  Dinner     2  0.139780
 4         24.59  3.61     No   Sun  Dinner     4  0.146808
 ..          ...   ...    ...   ...     ...   ...       ...
 239       29.03  5.92     No   Sat  Dinner     3  0.203927
 240       27.18  2.00    Yes   Sat  Dinner     2  0.073584
 241       22.67  2.00    Yes   Sat  Dinner     2  0.088222
 242       17.82  1.75     No   Sat  Dinner     2  0.098204
 243       18.78  3.00     No  Thur  Dinner     2  0.159744
 
 [244 rows x 7 co

In [59]:
grouped.agg({'tips_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tips_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [60]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

C:\Users\William Costa\AppData\Local\Temp\ipykernel_11092\2496195394.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tips.groupby(['day', 'smoker'], as_index=False).mean()


,day,smoker,total_bill,tip,size,tips_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply: General split-apply-combine